In [8]:
# import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
mpl.rcParams["figure.dpi"] = 150
import seaborn as sns
import os

# enable copy on write (default in pandas 3.0)
pd.options.mode.copy_on_write = True

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [10]:
def read_merged(weather, year):
    return pd.read_csv('../merged/merged_{}_{}.csv'.format(weather, year))

In [15]:
meso = [read_merged("meso", year) for year in range(2015, 2023)]

In [16]:
data = pd.concat(meso)
del meso

In [17]:
del data['Unnamed: 0']
del data['index']
data['TVS_max'] = 1*(data['TVS_max'] == 'Y')
data['DATE'] = pd.to_datetime(data['DATE'])
data['Month'] = data['DATE'].dt.month

In [18]:
data['y'] = 0

data.loc[data.power_outage == True, 'y']=1

In [19]:
all_features =([data.columns[1], 
                data.columns[2]] +
                data.columns[4:13].tolist() +
                [data.columns[16]])

In [20]:
all_features

['LAT_mean',
 'LON_mean',
 'LL_ROT_VEL_max',
 'LL_DV_max',
 'LL_BASE_max',
 'DEPTH_KFT_max',
 'DPTH_STMRL_max',
 'MAX_RV_KFT_max',
 'MAX_RV_KTS_max',
 'TVS_max',
 'MSI_max',
 'Month']

In [21]:
meso_train, meso_test = train_test_split(data.copy(),
                                              shuffle=True,
                                              random_state=123,
                                              test_size=.2,
                                              stratify=data.y.values)

In [22]:
meso_tt, meso_val = train_test_split(meso_train.copy(),
                                              shuffle=True,
                                              random_state=123,
                                              test_size=.2,
                                              stratify=meso_train.y.values)

In [27]:
outage = meso_tt[meso_tt['power_outage']==True]
no_outage = meso_tt[meso_tt['power_outage']==False]
no_outage= no_outage.sample(n=len(outage), random_state=101)
meso_tt_balanced = pd.concat([outage,no_outage],axis=0)

In [23]:
n_splits = 5

kfold = StratifiedKFold(n_splits,
                           shuffle=True,
                           random_state=498)

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
from sklearn.svm import SVC

In [33]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
'kernel': ['poly', 'rbf', 'sigmoid']}

In [34]:
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose=15, n_jobs=6,
                          cv = 3)


grid.fit(meso_tt_balanced[all_features], meso_tt_balanced.y)

Fitting 3 folds for each of 75 candidates, totalling 225 fits
